**A simple bandit algorithm**

*Initialize, for a = 1 to k:*

   $$Q(a) \leftarrow 0$$
   $$N(a) \leftarrow 0$$

*Repeat forever*:

$$A \leftarrow
\begin{cases}
argmax_a Q(a), & \text{with probability } 1 - \epsilon \text{ (breaking ties randomly)}\\
\text{a random action}, & \text{with probability } \epsilon \\
\end{cases}\\
R \leftarrow bandit(A)\\
N(A) \leftarrow N(A) + 1\\
Q(A) \leftarrow Q(A) + \cfrac{1}{N(A)}[R - Q(A)]$$


In [2]:
type Action = Int
type Reward = Int
type NthTry = Int
type Estimate = Double
type Epsilon = Double

trait SimpleBanditAlgorithm {

  def epoch = (1 to 2000)
  def eps: Epsilon = 0.1
  def nActions = 10

  import scala.util.Random
  def bandit(a: Action): Reward = a match {
    case 0 => 5 //Random.nextInt(20)
    case 5 => 98 //Random.nextInt(maxReward)
    case 7 => 14
    case 9 => 70 //Random.nextInt(30)  
    case _ => 60 //Random.nextInt(maxReward)
  }

  def randomAction: Action = Random.nextInt(nActions)
  def distribution(eps: Epsilon): Boolean = Random.nextDouble() < eps

  def argmax(values: Map[Action, Estimate]): Action = if (values.nonEmpty){
    values.iterator.maxBy(_._2)._1
  } else randomAction
    
  final case class State(n: NthTry = 0, accR: List[Reward] = List.empty, q: Map[Action, Estimate] = Map.empty)

  def step(state: State): State = {
    val action = 
      if (distribution(eps)) randomAction else argmax(state.q)
    val reward = bandit(action)
    val nUpdate = state.n + 1
    val previousEstimate = state.q.getOrElse(action, 0.0)
    val updatedEstimate = 
      previousEstimate + (reward - previousEstimate) / nUpdate
    val qUpdate = state.q.updated(action, updatedEstimate)
    State(nUpdate, reward :: state.accR, qUpdate)
  }

  val results = epoch
    .scanLeft(State())((state, _) => step(state))

}

defined type Action
defined type Reward
defined type NthTry
defined type Estimate
defined type Epsilon
defined trait SimpleBanditAlgorithm

The average reward for different $\epsilon$:

In [3]:
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.0`

import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._

plotly.JupyterScala.init()

def algByEps(epsilon: Epsilon) = new SimpleBanditAlgorithm {
  override def eps = epsilon
}

val plot = Seq(0.1, 0.01, 0) map { eps =>
  val (x, y) = algByEps(eps).results.map(s => s.n -> s.accR.sum / (s.accR.size + 1)).unzip
  Scatter(x, y, name = eps.toString)  
}

def drawPlot(): Unit = plot.plot()

import $ivy.$                                             


import plotly._

import plotly.element._

import plotly.layout._

import plotly.JupyterScala._


defined function algByEps
plot: Seq[plotly.Scatter] = List(
  Scatter(
    Some(
      Doubles(
        Vector(
          0.0,
          1.0,
          2.0,
          3.0,
          4.0,
          5.0,
          6.0,
...
defined function drawPlot

In [4]:
drawPlot()

Same Plot rendered with Vegas library:

In [7]:
import $ivy.`org.vegas-viz::vegas:0.3.6`

import vegas._
import vegas.render.HTMLRenderer._

implicit val displayer: String => Unit = publish.html(_)


val vegasPlotData = for {
  eps <- Seq(0.1, 0.01, 0)
  s <- algByEps(eps).results
} yield Map(
  "eps" -> eps.toString, 
  "step" -> s.n, 
  "average reward" -> s.accR.sum / (s.accR.size + 1)
) 

Vegas("Plot")
  .withData(vegasPlotData)
  .encodeX("step", Quant)
  .encodeY("average reward", Quant)
  .encodeDetailFields(Field(field = "eps", dataType = Nominal))
  .encodeColor(
       field = "eps",
       dataType = Nominal,
       legend = vegas.Legend(orient = "left", title = "epsilon"))
  .mark(vegas.Line)
  .show

<iframe id="frame-vegas-c0ab6635-3dfd-41a7-896c-b5a9af41d163" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="http://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="http://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="http://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "line",
 "encoding" : {
 "x" : {
 "field" : "step",
 "type" : "quantitative"
 },
 "y" : {
 "field" : "average reward",
 "type" : "quantitative"
 },
 "color" : {
 "legend" : {
 "title" : "epsilon",
 "orient" : "left"
 },
 "field" : "eps",
 "type" : "nominal"
 },
 "detail" : {
 "field" : "eps",
 "type" : "nominal"
 }
 },
 "description" : "Plot",
 "data" : {
 "values" : [
 {
 "eps" : "0.1",
 "step" : 0,
 "average reward" : 0
 },
 {
 "eps" : "0.1",
 "step" : 1,
 "average reward" : 30
 },
 {
 "eps" : "0.1",
 "step" : 2,
 "average reward" : 40
 },
 {
 "eps" : "0.1",
 "step" : 3,
 "average reward" : 45
 },
 {
 "eps" : "0.1",
 "step" : 4,
 "average reward" : 48
 },
 {
 "eps" : "0.1",
 "step" : 5,
 "average reward" : 50
 },
 {
 "eps" : "0.1",
 "step" : 6,
 "average reward" : 51
 },
 {
 "eps" : "0.1",
 "step" : 7,
 "average reward" : 52
 },
 {
 "eps" : "0.1",
 "step" : 8,
 "average reward" : 53
 },
 {
 "eps" : "0.1",
 "step" : 9,
 "average reward" : 54
 },
 {
 "eps" : "0.1",
 "step" : 10,
 "average reward" : 54
 },
 {
 "eps" : "0.1",
 "step" : 11,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 12,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 13,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 14,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 15,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 16,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 17,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 18,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 19,
 "average reward" : 54
 },
 {
 "eps" : "0.1",
 "step" : 20,
 "average reward" : 54
 },
 {
 "eps" : "0.1",
 "step" : 21,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 22,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 23,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 24,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 25,
 "average reward" : 55
 },
 {
 "eps" : "0.1",
 "step" : 26,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 27,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 28,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 29,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 30,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 31,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 32,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 33,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 34,
 "average reward" : 56
 },
 {
 "eps" : "0.1",
 "step" : 35,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 36,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 37,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 38,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 39,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 40,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 41,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 42,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 43,
 "average reward" : 57
 },
 {
 "eps" : "0.1",
 "step" : 44,
 "average reward" : 58
 },
 {
 "eps" : "0.1",
 "step" : 45,
 "average reward" : 58
 },
 {
 "eps" : "0.1",
 "step" : 46,
 "average reward" : 58
 },
 {
 "eps" : "0.1",
 "step" : 47,
 "average reward" : 58
 },
 {
 "eps" : "0.1",
 "step" : 48,
 "average reward" : 58
 },
 {
 "eps" 

import $ivy.$                           


import vegas._

import vegas.render.HTMLRenderer._


displayer: String => Unit = <function1>
vegasPlotData: Seq[Map[String, Any]] = List(
  Map("eps" -> 0.1, "step" -> 0, "average reward" -> 0),
  Map("eps" -> 0.1, "step" -> 1, "average reward" -> 30),
  Map("eps" -> 0.1, "step" -> 2, "average reward" -> 40),
  Map("eps" -> 0.1, "step" -> 3, "average reward" -> 45),
  Map("eps" -> 0.1, "step" -> 4, "average reward" -> 48),
  Map("eps" -> 0.1, "step" -> 5, "average reward" -> 50),
  Map("eps" -> 0.1, "step" -> 6, "average reward" -> 51),
  Map("eps" -> 0.1, "step" -> 7, "average reward" -> 52),
  Map("eps" -> 0.1, "step" -> 8, "average reward" -> 53),
  Map("eps" -> 0.1, "step" -> 9, "average reward" -> 54),
  Map("eps" -> 0.1, "step" -> 10, "average reward" -> 54),
...